# TD - PICSEL : Champ de speckles et effets chromatiques


# Introduction

Le but de ce TD est de reproduire, par simulation, une observation visuelle simple : observer les motifs produits par la lumière d'un flash de téléphone.  

### Procédure expérimentale

1. Un téléphone est placé à quelques mètres de l'observateur avec son **flash allumé**.  
2. L'observateur regarde directement le flash et remarque l'apparition de **structures particulières colorées** appelées **speckles**.  
3. On répète l'observation avec un **diaphragme placé devant l'œil**, ce qui modifie la taille de la pupille et la perception des speckles.  

Ces motifs résultent d'interférences aléatoires de la lumière cohérente et dépendent de la diffraction et de la taille de la pupille.

## Objectif

L'objectif de ce TD est de **reproduire ces effets visuellement à l'aide de simulations basées sur l'optique de Fourier**.  

### Sujets abordés

Nous aborderons :  

- **Principe basique de la génération d'une image au foyer d'une lentille** – Point Spread Function (PSF)  
- **Limite de diffraction** et son influence sur la résolution  
- **Chromaticité de la PSF** suivant la longueur d'onde  
- **Aberrations et speckles**
- **Chromaticité des aberrations** et impact sur l'image perçue 

## Méthodologie

Ce Jupyter Notebook fournit une librairie de fonctions utilitaires pour ce TD, afin de se concentrer sur l'étude des speckles sans perdre de temps à coder les fonctions de base.  

### Fonctions principales disponibles

- **Transformée de Fourier 2D centrée**  
  `tf(arr)` : calcule la transformée de Fourier centrée d’un tableau 2D.

- **Création et manipulation de la pupille**  
  - `make_pupil(Rpx, nPx)` : génère une pupille circulaire de rayon `Rpx` pixels dans une matrice `nPx × nPx`.  
  - `pad(arr, pad_width)` : ajoute un zero-padding à un tableau 2D.

- **Gestion des couleurs et affichage**  
  - `spectral_to_rgb_image(data, light_colors)` : combine plusieurs cartes d’intensité spectrales en une seule image RGB.  
  - `show_rgb(rgb_image, title)` : affiche une image RGB dans le notebook avec Matplotlib.
  - `show_spectral_psf(psfs, wvl, vmin=None)` : affiche une PSF à une longueur d'onde

- **Simulation de phase aléatoire**  
  `generate_power_law_phase_screen(N, alpha)` : génère un écran de phase 2D avec un spectre en loi de puissance (`PSD ~ f^(-alpha)`).

- **Couleurs prédéfinies**  
  `light_colors` : dictionnaire associant longueurs d’onde visibles (en nm) à leurs valeurs RGB pour la visualisation.



## Étape 1 – Génération et visualisation d'une pupille

Dans cette première étape, nous allons créer une **pupille circulaire**.

### Objectifs

- Générer une pupille circulaire.
- Visualiser la pupille.

### Instructions

1. **Créer une pupille circulaire** avec la fonction dédiée.

2. **Visualiser la pupille** avec Matplotlib. 

In [1]:
import numpy
from utils import *
import matplotlib.pyplot as plt
%matplotlib qt

In [12]:
# Define pupil
Npx = 256
Rpx = Npx // 10
pupil = make_pupil(Rpx,Npx)

# Plot
plt.figure()
plt.imshow(pupil)
plt.show()

## Étape 2 – Formation d'image : calcul de la PSF

Dans cette étape, nous allons passer de la pupille à l'image au foyer d'une lentille en calculant la **Point Spread Function (PSF)**.

### Objectifs

- Comprendre le lien entre la **pupille** et l'**image d’un point lumineux**.  
- Visualiser la PSF et observer la **limite de diffraction**.  
- Apprendre l’importance du **sampling** et du **zero-padding** pour correctement représenter la PSF.

### Théorie rapide

1. **Lien pupille → image** :  
   L’image d’un point lumineux après passage par une lentille est donnée par la **transformée de Fourier de la pupille**.  
   - Donner la formule qui relie la taille caractéristique de la PSF avec la longueur d'onde et le diamètre de la pupille.
   - Donner la formule qui relie le champ en plan pupille et la PSF.

2. **Sampling et zero-padding** :  
   - Pour simuler l'effet d'une lentille on va utiliser l'algorithme de la FFT (Fast-Fourier-Transform). Dans ce cas, le **nombre de pixels par $\lambda/D$** dépend du rapport :  
     $$\frac{N}{D}$$ 
     où \(D\) est le diamètre de la pupille en pixels et \(N\) la taille du tableau en pixels.  
   - Pour respecter le **critère de Shannon**, il faut au moins **2 pixels par $\lambda/D$**.  
   - Le **zero-padding** permet d’augmenter \(N\) et donc d’augmenter le nombre de pixels par $\lambda/D$ sans changer la pupille réelle.

### Instructions

1. **Calculer la PSF** à partir de la pupille crée précédemment.
   - Dans le cas d'un sampling à Shannon
   - Dans le cas d'un sampling à 10 x shannon

2. **Visualiser les PSF** générées, ne pas hésiter à utiliser une échelle logarithmique.


In [13]:
# Shannon
sampling = Npx//2
# sampling p = 5
p = 5
sampling = int((p*Npx-Npx)/2)

pupil_pad = pad(pupil,sampling)
# Plot
plt.figure()
plt.imshow(pupil_pad)
plt.show()

psf = np.abs(tf(pupil_pad))**2

# Plot
plt.figure()
plt.imshow(psf**0.2)
plt.show()

## Étape 3 – Aberrations de front d’onde

Dans cette étape, nous allons introduire des **aberrations de front d’onde** et étudier leur impact sur la PSF.  
Les aberrations modifient la phase du champ complexe dans le plan pupille et dégradent la qualité de l’image.

### Objectifs

- Comprendre ce qu’est une aberration de front d’onde.
- Générer une aberration de **phase**.
- Savoir comment inclure une aberration dans le calcul de la PSF.
- Observer l’effet des aberrations sur la PSF (élargissement, asymétrie, speckles).

### Instructions

1. **Générer une aberration de phase**  à l'aide de la fonction dédiée. Comprendre l'effet du paramètre alpha.

2. **Inclure cette abberations dans la formation d'image**
   - Modifier le champ en plan pupille pour prendre en compte cette aberration.
   - Former la PSF correspondante, identifier ce que l'on appelle le champ de "speckle".


In [14]:
phase = generate_power_law_phase_screen(Npx, 2) # Power law parameter weight energy on low frequencies versus high frequencies

# Plot
plt.figure()
plt.imshow(phase,cmap='seismic')
plt.show()

p = 20
sampling = int((p*Npx-Npx)/2)
EM = pupil * np.exp(1j * phase)
EM_pad = pad(EM,sampling)

psf_aberrante = np.abs(tf(EM_pad))**2
plt.figure()
plt.imshow(psf_aberrante**0.2)
plt.show()

## Étape 4 – Chromaticité de la PSF et des aberrations

Dans cette étape, nous étudions l’effet de la **longueur d’onde** sur la PSF.  
La chromaticité intervient à **deux niveaux distincts** :
1. Le **scaling de diffraction** de la PSF.
2. Le **scaling des aberrations de phase**.

### Objectifs

- Comprendre l’origine de la chromaticité de la PSF.
- Identifier les deux effets chromatiques distincts.
- Relier **OPD (Optical Path Difference)** et **phase**.
- Simuler une PSF chromatique réaliste.

### 1. Chromaticité de la diffraction

La taille caractéristique de la PSF est proportionnelle à la longueur d’onde. 

1. **Générer une PSF polychromatique**. (indice : faire en sorte que la PSF la plus étroite soit échantillonnée à Shannon et jouer sur le zero-padding), le format de sortie doit être:

    dict {wavelength_nm: NxN numpy array}

2. **Visualiser la PSF polychromatique à l'aide de la fonction dédiée**.


### 2. Chromaticité des aberrations

Dans l’œil (et dans de nombreux systèmes optiques), les aberrations sont bien décrites par une **OPD constante** (en mètres).

1. **Donner le lien entre OPD et phase**

2. **Ajouter cet effet dans la simulation de PSF chromatique**.

3. **Visualiser la PSF polychromatique**, comparer à la PSF à une longueur d'onde donnée.





In [22]:
# Dictionnary
psfs = {}

for wvl in light_colors:
    print(wvl)
    # Fabriquer PSF à wvl
    p_wvl = 2 * wvl / 420
    sampling_wvl = int((p_wvl*Npx-Npx)/2)
    phase_wvl = 0*phase * 420 / wvl
    pupil_wvl_pad = np.pad(pupil * np.exp(1j * phase_wvl),sampling_wvl)
    psf_wvl = np.abs(tf(pupil_wvl_pad))**2
    # Crop tableaux
    N_crop = Npx
    psf_wvl = psf_wvl[psf_wvl.shape[0]//2-N_crop:psf_wvl.shape[0]//2+N_crop,psf_wvl.shape[0]//2-N_crop:psf_wvl.shape[0]//2+N_crop]
    # sauvegarder PSF
    psfs[wvl] = psf_wvl

# Plot PSF
plt.figure()
plt.subplot(121)
plt.imshow(psfs[420]**0.2)
plt.subplot(122)
plt.imshow(psfs[700]**0.2)
plt.show()


700
610
580
530
500
470
420


In [23]:
psf_polychromatique = spectral_to_rgb_image(psfs, light_colors)

In [24]:
psf_polychromatique.shape

(512, 512, 3)

In [25]:
show_rgb(psf_polychromatique,'PSF RGB')